In [1]:
import pandas as pd
import numpy as np
import re
from pptx import Presentation
# import pathlib
from pptx.chart.data import CategoryChartData
from pptx.dml.color import RGBColor
from pptx.enum.chart import XL_CHART_TYPE, XL_LABEL_POSITION
from pptx.enum.text import MSO_ANCHOR
from pptx.enum.text import MSO_VERTICAL_ANCHOR
from pptx.enum.text import PP_ALIGN
from pptx.oxml.xmlchemy import OxmlElement
from pptx.util import Pt, Cm, Inches
from datetime import datetime as dt

In [4]:
global prs,path_to_template,path_to_excel
path_to_template = 'D:\Desktop\钟鼎资本\代码\自动化PPT模版.pptx'
path_to_excel = 'D:\Desktop\钟鼎资本\代码\自动化PPT所需数据.xlsx'
path_to_medicine_white = 'D:\Desktop\钟鼎资本\代码\医疗机构白名单.xlsx'
path_to_project = r'D:\Desktop\钟鼎资本\代码\全市场跟踪\历史融资数据.xlsx'

In [2]:
def abstract_time(db,start_time,end_time):
    db['日期2'] = pd.to_datetime(db['日期'])  # 将日期由str格式转换为datetime64格式
    # 得到所需源数据！！！
    db_2 = db[(db['日期2'] >= start_time) & (db['日期2'] <= end_time)]
    return db_2
def cover(content):
    #prs.slide_layouts[0]获取ppt自带的第一种版式
    slide = prs.slides.add_slide(prs.slide_layouts[0])
    #添加一个文本框
    textBox = slide.shapes.add_textbox(left=Cm(1.27), top=Cm(7.43), width=Cm(22.93), height=Cm(4.08))
    # 获取文本框文字对象
    tf = textBox.text_frame
    # 文本框样式调整
    # tf.margin_bottom = Cm(0.1)  # 下边距
    # tf.margin_left = 0  # 左边距
    tf.vertical_anchor = MSO_VERTICAL_ANCHOR.MIDDLE  # 垂直方式：底端对齐
    tf.word_wrap = True  # 文本框的文字自动对齐
    # 设置内容
    tf.paragraphs[0].text = content
    # 字体样式调整
    tf.paragraphs[0].alignment = PP_ALIGN.LEFT  # 对齐方式
    tf.paragraphs[0].font.name = '华文楷体'  # 字体名称
    tf.paragraphs[0].font.bold = True  # 是否加粗
    # tf.paragraphs[0].font.italic = True  # 是否斜体
    tf.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)  # 字体颜色
    tf.paragraphs[0].font.size = Pt(28)  # 字体大小

def part1(db,area,time_period):
    #1-处理一级市场热点所需数据（去重，添加柱状图的赛道区分）
    #db = db_cn
    #area = '中国'
    #处理属于多个赛道的情况
#     db = multi_indu(db)
    db1 = db.drop_duplicates(subset=['名称','一级赛道-新'], keep='first')  # 按投资机构事件名称和一级赛道去重
#     indus_for_barchart = pd.read_excel(path_to_excel, sheet_name='一级市场热点')  # import作柱状图的行业重分类，之后修改
#     db1 = db1.merge(indus_for_barchart, on=['一级赛道', '二级赛道', '三级赛道'], how='left')  # merge重分类到db
    #去除ipo等5类
    db1_new = db1[~db1['轮次'].isin(['IPO上市','私有化','并购','定向增发','股权转让'])]
    # 2-作一级市场热点柱状图
    barchart_data = db1_new['一级赛道-新'].value_counts()  # 按重分类行业进行统计，并降序排序，#得到做柱状图所需数据
    if area == '中国':
        title_content = '国内一级市场投融资热点'+str(time_period)
    else:
        title_content = '国外一级市场投融资热点'+str(time_period)
    get_barchart_slide(barchart_data,title_content)

    #3-所有赛道展开表格
    # industry_list = list(set(db1['柱状图行业划分']))
    #处理金额项目是否是亿元
    db2_new = get_indus_table_data(db1_new)
    #检查
    db2_new = check_whitelist(db2_new,area)
    #先得到汇总表格
    industry_list = list(barchart_data.index)  # 按照降序排序的重分类
    #if area == '中国':
    for indus in industry_list:
        # content = str(indus)+'再打开~top20公司'
        # get_indus_table_slide(indus,db2,content)
        content = str(indus) + '再打开~top20公司'+str(time_period)
        get_indus_table_slide(indus, db2_new, content)
    #else:
    #    db2_new['柱状图行业划分'] = db2_new['柱状图行业划分'].astype('category').cat.set_categories(industry_list)
    #    db2_new = db2_new.sort_values(by=['柱状图行业划分','投资金额(亿人民币)'],ascending=[True,False])
    #    content ='各赛道再打开'+str(time_period)
    #    get_ALL_indus_table_slide(db2_new, content)

def white_expand(db,area,time_period):
    #是否属于top20公司
    top20_list = check_non_top20(db)

    #4-白名单投资机构数据筛选，db3是有白名单机构投资的公司
    db3 = match_invest_with_whitelist(db,area)
    db3 = db3.reset_index(drop=True)

    #获取白名单列表
    white_table = pd.read_excel(path_to_excel,sheet_name='国内白名单投资机构')
    white_table = list(white_table.白名单投资机构)
    #处理多个投资人
    db['本轮投资机构'] = db['投资机构(总)']
    # db = db.drop_duplicates(subset=['项目','轮次'])
    db = db.drop_duplicates()
    db = multi_invest(db)
#     db.to_excel(r'D:\Desktop\db1.xlsx')
    db = db.reset_index(drop=True)
    non_white = []
    for i in range(0,len(db)):
        for j in range(0,len(white_table)):
            flag = 0
            if white_table[j] in db.loc[i,'投资机构(总)']:
                db.loc[i,'投资机构(总)'] = white_table[j]
                flag = 1
                break
        if flag == 0:#如果不在白名单上
            non_white.append(i)
    db = db.drop(non_white)
    db = db.reset_index(drop=True)
    #投资机构有哪些,且按照出手次数从大到小排序
    invest_num = db.groupby('投资机构(总)')['项目'].count()
    invest = invest_num.sort_values(ascending=False).index
#     db.to_excel(r'D:\Desktop\db2.xlsx',index=False)
    total_data = pd.read_excel('D:\Desktop\钟鼎资本\代码\全市场跟踪\历史融资数据.xlsx')
    total_data['日期'] = pd.to_datetime(total_data['日期'])
    total_data['日期'] = total_data['日期'].apply(date_to_ymd)
    for i in invest:
        db_invest = db[db['投资机构(总)']==i]
        content = time_period+'-'+str(i)
        get_whiteinvest_table_slide3(db_invest,content,total_data)
        
#赛道再展开ppt       
def get_indu_reexpand_slide(whiteinvest_list_data,content,indu,area):
    #获得投资金额(亿人民币)这一列
#     whiteinvest_list_data = get_indus_table_data(whiteinvest_list_data)
#     whiteinvest_list_data = whiteinvest_list_data.sort_values(by =['项目','投资金额(亿人民币)'],ascending=False)

    if indu == '软件':
        df2 = whiteinvest_list_data[['项目','软件的三级label','业务','城市','轮次','投资模糊金额(数值加金额)','投资机构(总)','A轮及以前投资机构','B轮投资机构']].values
        df2 = np.insert(df2, 0, values=np.array([['项目','钟鼎标签','业务','城市','轮次','投资金额(亿￥)','最新一轮投资机构','A轮及以前投资机构','B轮投资机构']]), axis=0)
        col_index = [2,6,7,8]
    else:
        df2 = whiteinvest_list_data[['项目','业务','城市','轮次','投资模糊金额(数值加金额)','投资机构(总)','A轮及以前投资机构','B轮投资机构']].values
        df2 = np.insert(df2, 0, values=np.array([['项目','业务','城市','轮次','投资金额(亿￥)','最新一轮投资机构','A轮及以前投资机构','B轮投资机构']]), axis=0)
        col_index = [1,5,6,7]
    font_size = 11
    # length = len(df2)
    # print('数据总长度为',length)
    # height = 0
    # pages = 0
    # max_height = []
    # num_per_page = []
    # flag = 0
    #找到需要几页，每页能容纳几个记录
    # for i in range(0,length):
    #     for j in col_index:
    #         if j == 1 or j == 2:
    #             max_height.append(cal_height(0.46541,0.269889,font_size,5,df2[i][j]))
    #         else:
    #             max_height.append(cal_height(0.46541,0.269889,font_size,2.712,df2[i][j]))
    #     max_height.sort()
    #     height = height+max_height[-1]
    #     if height <= 5.833:
    #         if i == length-1 :#如果全部记录长度连一页都不够
    #             if pages == 0:
    #                 pages = 1
    #                 num_per_page.append(length)
    #                 # print('188',str(i)+'行',height)
    #             else:#最后一页长度不满一页
    #                 pages = pages + 1
    #                 num_per_page.append(i-flag+1)
    #                 # print('192',str(i)+'行',height)
    #     else:
    #         # print('198',str(i)+'行',height)
    #         height = 0.92#0.128228为第一行的高度
    #         if i == length-1:
    #             num_per_page.append(i-flag)
    #             num_per_page.append(1)
    #             pages = pages+1
    #         else:
    #             num_per_page.append(i-flag)
    #             flag = i
    #         pages = pages + 1
    #         #除去表头，每页还能容纳几行,第一页是算上了表头，之后几页没算上
    # # print('分页结果为',num_per_page)
    # # print('共多少页',pages)
    # for j in range(1,len(num_per_page)):
    #     num_per_page[j] = num_per_page[j]+1
    num_per_page = 5
    pages_f = lambda x: (x // num_per_page) + 1 if x % num_per_page > 0 else x // num_per_page
    pages = pages_f(len(df2)-1)
    area = '中国'

    for i in range(0, pages):
        slide = prs.slides.add_slide(prs.slide_layouts[1])
        #content = '白名单投资机构投资事件'
        content1 = content+'('+str(i+1)+'/'+str(pages)+')'
        add_title(slide, content1)
        rows,cols = min(len(df2),num_per_page+1),df2.shape[1]

        # 设置表格位置和大小
        if indu == '软件':#软件业要单独设计表格，因为多了一列
            left, top, width, height = Cm(0.1), Cm(2.7), Cm(24.46), Cm(19.3)
            # 表格行列数，和大小
            shape = slide.shapes.add_table(rows, cols, left, top, width, height)
            table = shape.table
            for i in range(0,cols):
                if df2[0][i] == '项目':
                    table.columns[i].width = Cm(1.7)
                elif df2[0][i] == '业务':
                    table.columns[i].width = Cm(5)
                elif df2[0][i] == '城市' or df2[0][i] == '轮次' :
                    table.columns[i].width = Cm(1.5)
                elif df2[0][i] == '投资金额(亿￥)':
                    table.columns[i].width = Cm(2.1)
                elif df2[0][i] == '最新一轮投资机构' or df2[0][i] == 'A轮及以前投资机构' or df2[0][i] == 'B轮投资机构' :
                    table.columns[i].width = Cm(3.72)
                elif df2[0][i] == '钟鼎标签':
                    table.columns[i].width = Cm(2.2)
                else:
                    table.columns[i].auto_size = True
                    table.columns[i].fit_text = True
        else:
            left, top, width, height = Cm(0.1), Cm(2.7), Cm(24.46), Cm(19.65)
            # 表格行列数，和大小
            shape = slide.shapes.add_table(rows, cols, left, top, width, height)
            table = shape.table
            for i in range(0,cols):
                if df2[0][i] == '项目':
                    table.columns[i].width = Cm(1.7)
                elif df2[0][i] == '业务':
                    table.columns[i].width = Cm(6)
                elif df2[0][i] == '城市' or df2[0][i] == '轮次' :
                    table.columns[i].width = Cm(1.5)
                elif df2[0][i] == '投资金额(亿￥)':
                    table.columns[i].width = Cm(2.1)
                elif df2[0][i] == '最新一轮投资机构' or df2[0][i] == 'A轮及以前投资机构' or df2[0][i] == 'B轮投资机构' :
                    table.columns[i].width = Cm(4.14)
                else:
                    table.columns[i].auto_size = True
                    table.columns[i].fit_text = True


        start_merge = 1
        end_merge = 1

        # 赋值，调格式
        for row in range(0,rows):
            table.rows[row].height = Cm(0.92)
            for col in range(cols):
                if row == 0:
                    table.cell(row,col).text = str(df2[0][col])
                else:
                    if col == 0:#如果是第一列项目列，要判断要不要合并
                        if df2[1][col] == table.cell(row-1,col).text:#如果两个项目相同，合并单元格
                            end_merge = row
                        else:
                            cell1 = table.cell(start_merge,0)
                            cell2 = table.cell(end_merge,0)
                            cell1.merge(cell2)
                            table.cell(row, col).text = str(df2[1][col])
                            start_merge = row
                            end_merge = row
                    elif df2[0][col] == '城市':#如果是城市列，要取出“市”
                        if str(df2[1][col]).find('市') != -1:
                            table.cell(row, col).text = str(df2[1][col])[0:str(df2[1][col]).find('市')]
                        else:
                            table.cell(row, col).text = str(df2[1][col])

                    # elif df2[0][col] == '投资金额(亿￥)':#如果是投资金额列
                    #     if pd.isnull(df2[1][col]) == False:
                    #         if df2[1][col] == 0:
                    #             table.cell(row, col).text = '/'#如果为零，用斜杆替换
                    #         else:
                    #             table.cell(row, col).text = format(df2[1][col],'.1f')
                    #     else:
                    #         table.cell(row, col).text = '/'#如果空值，用斜杆替换

                    elif df2[0][col] == '最新一轮投资机构' or df2[0][col] == 'A轮及以前投资机构' or df2[0][col] == 'B轮投资机构':
                        red_signal(df2[1][col],table,row,col,area,indu)

                    else:#其他列
                        if pd.isnull(df2[1][col]) == False:
                            table.cell(row, col).text = str(df2[1][col])
                        else:
                            table.cell(row, col).text = '/'
                # 设置文字大小
                table.cell(row, col).text_frame.paragraphs[0].font.size = Pt(font_size)
                # 设置字体
                table.cell(row, col).text_frame.paragraphs[0].font.name = '华文楷体'
                # table.cell(row, col).text_frame.paragraphs[0].font.name = 'Corbel'
                # 设置文字颜色
                # table.cell(row, col).text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
                # 设置文字左右对齐
                table.cell(row, col).text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                # 设置文字上下对齐
                table.cell(row, col).vertical_anchor = MSO_ANCHOR.MIDDLE
                # 设置背景为填充
                # table.cell(row, col).fill.solid()

            if row > 0:
                df2 = np.delete(df2, 1, 0)#删除第一行

#白名单机构展开ppt       
def get_whiteinvest_table_slide3(whiteinvest_list_data,content,total_data):
    #获得投资金额(亿人民币)这一列
    whiteinvest_list_data = get_indus_table_data(whiteinvest_list_data)
    whiteinvest_list_data = check_whether_first(whiteinvest_list_data,total_data)
    # whiteinvest_list_data = sort_fuzzy_amount(whiteinvest_list_data)
    whiteinvest_list_data = whiteinvest_list_data.sort_values(by =['投资金额(亿人民币)'],ascending=False)
    # whiteinvest_list_data['序号'] = whiteinvest_list_data['投资金额(亿人民币)'].rank(method='first',ascending=False)

    df2 = whiteinvest_list_data[['项目','钟鼎标签','业务','城市','轮次','投资模糊金额(数值加金额)','本轮投资机构','A轮及以前投资机构','B轮投资机构','首次进入/追加投资']].values

    num_per_page = 4
    pages_f = lambda x: (x // num_per_page) + 1 if x % num_per_page > 0 else x // num_per_page
    pages = pages_f(len(df2))
    df2 = np.insert(df2, 0, values=np.array([['项目','钟鼎标签','业务','城市','轮次','投资金额(亿￥)','最新一轮投资机构','A轮及以前投资机构','B轮投资机构','首次进入/追加投资']]), axis=0)
    for i in range(0, pages):
        slide = prs.slides.add_slide(prs.slide_layouts[1])
        #content = '白名单投资机构投资事件'
        num_invest = len(whiteinvest_list_data)
        first_invest = len(whiteinvest_list_data[whiteinvest_list_data['首次进入/追加投资']=='首次进入'])
        add_invest = len(whiteinvest_list_data[whiteinvest_list_data['首次进入/追加投资']=='追加投资'])
        care_invest = len(whiteinvest_list_data[whiteinvest_list_data['标签（一级）']!='非关注领域'])
        content1 = content+'共出手{0}次，首次{1}次，追加{2}次；钟鼎关注领域出手{3}次'.format(num_invest,first_invest,add_invest,care_invest)+'('+str(i+1)+'/'+str(pages)+')'
        add_title(slide, content1)
        # 设置表格位置和大小
        left, top, width, height = Cm(0.05), Cm(2.7), Cm(24.46), Cm(14.97)
        rows, cols = min(len(df2),num_per_page+1), df2.shape[1]
        # 表格行列数，和大小
        shape = slide.shapes.add_table(rows, cols, left, top, width, height)
        table = shape.table
        table.columns[0].width = Cm(1.7)
        table.columns[1].width = Cm(2)
        table.columns[2].width = Cm(4.3)
        table.columns[3].width = Cm(1.5)
        table.columns[4].width = Cm(1.5)
        table.columns[5].width = Cm(1.83)
        table.columns[6].width = Cm(3.4)
        table.columns[7].width = Cm(3.3)
        table.columns[8].width = Cm(3.3)
#         table.columns[9].width = Cm(3)
        area = '中国'
        # 赋值，调格式
        start_merge = 1
        end_merge = 1
        for row in range(rows):
            table.rows[row].height = Cm(0.92)
            for col in range(cols):
                if row == 0:
                    table.cell(row,col).text = str(df2[0][col])
                else:
                    if col == 0:#如果是第一列项目列，要判断要不要合并
                        if df2[1][col] == table.cell(row-1,col).text:#如果两个项目相同，合并单元格
                            end_merge = row
                        else:
                            cell1 = table.cell(start_merge,0)
                            cell2 = table.cell(end_merge,0)
                            cell1.merge(cell2)
                            table.cell(row, col).text = str(df2[1][col])
                            start_merge = row
                            end_merge = row
                    # elif col == 5:#如果是投资金额列
                    #     if pd.isnull(df2[1][col]) == False:
                    #         if df2[1][col] == 0:
                    #             table.cell(row, col).text = '/'#如果为零，用斜杆替换
                    #         else:
                    #             table.cell(row, col).text = format(df2[1][col],'.1f')
                    elif col == 3:#如果是城市列，要取出“市”
                        if str(df2[1][col]).find('市') != -1:
                            table.cell(row, col).text = str(df2[1][col])[0:str(df2[1][col]).find('市')]
                        else:
                            table.cell(row, col).text = str(df2[1][col])
                    else:#其他列
                        if pd.isnull(df2[1][col]) == False:
                            table.cell(row, col).text = str(df2[1][col])
                            if df2[1][col] == '首次进入':
                                table.cell(row, col).text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 0, 0)
                        else:
                            table.cell(row, col).text = '/'
            
                # 设置文字大小
                table.cell(row, col).text_frame.paragraphs[0].font.size = Pt(11)
                # 设置字体
                table.cell(row, col).text_frame.paragraphs[0].font.name = '华文楷体'
                # table.cell(row, col).text_frame.paragraphs[0].font.name = 'Corbel'
                # 设置文字颜色
                # table.cell(row, col).text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
                # 设置文字左右对齐
                table.cell(row, col).text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                # 设置文字上下对齐
                table.cell(row, col).vertical_anchor = MSO_ANCHOR.MIDDLE
                # 设置背景为填充
                # table.cell(row, col).fill.solid()

            if row > 0:
                df2 = np.delete(df2, 1, 0)#删除第一行
#标红函数
def red_signal(text,table,row,col,area,indu):
    tf = table.cell(row, col).text_frame.paragraphs[0]
    str_list = str(text)
    str_list = re.split(',|、|\n|，', str_list)
    for i in range(len(str_list)):
        if check_white(str_list[i],area,indu):
            red_run = tf.add_run()
            red_run.text = str_list[i]
            red_run.font.color.rgb = RGBColor(255, 0, 0)
        else:
            run = tf.add_run()
            run.text = str_list[i]
            run.font.color.rgb = RGBColor(0, 0, 0)
        if i + 1 < len(str_list):
            run = tf.add_run()
            run.text = ','
            run.font.color.rgb = RGBColor(0, 0, 0)
    
def check_non_top20(db):

    db1 = db.sort_values(by = '投资模糊金额(万)',ascending=False)
    db1 = db1.reset_index(drop=True)
    industry = list(set(db1['标签（一级）']))#重复元素在set中自动过滤
    db2 = pd.DataFrame()
    for j in industry:
        db_j = db1[db1['标签（一级）'] == j]
        #db_j['是否不属于top20'] = 1
        db_j = db_j.reset_index(drop=True)
        db_j_20 = db_j.loc[19:]
        #for i in db_j.index:
        #    if i<20:
        #        temp = db_j['是否不属于top20'][i]
        db2 = db2.append(db_j_20)
    top20_list = list(set(db2['项目']))
    return top20_list

#把投资机构（总）中有白名单投资机构的分拆
def match_invest_with_whitelist(db,area):
    if area == '中国':
        white_list = pd.read_excel(path_to_excel,sheet_name='国内白名单投资机构') #导入白名单机构list
    else:
        white_list = pd.read_excel(path_to_excel, sheet_name='海外白名单投资机构')  # 导入白名单机构list
    #db2['白名单投资机构'] = ''
    whitelist = list(set(white_list['白名单投资机构']))
    ##跟"投资机构"列匹配code,得到白名单机构的投资事件db3
    db3 = pd.DataFrame()
    for i in whitelist:
        for j in db.index:
            if pd.isnull(db['投资机构(总)'][j])==False:
                if i in db['投资机构(总)'][j]:
                    temp = db.loc[j]
                    temp['白名单投资机构'] = i
                    db3 = db3.append(temp)

    return db3

def get_indus_table_data(db1):
    db1['投资金额(亿人民币)'] = db1['投资模糊金额(万)']/10000
    #db2 = db1[['日期', '项目', '业务', '轮次', '投资金额(亿人民币)', '投资机构(总)','柱状图行业划分']]
    return db1

def add_title(slide,content):
    #slide = prs.slides.add_slide(prs.slide_layouts[1])
    textBox = slide.shapes.add_textbox(left=Cm(0.26), top=Cm(1.05), width=Cm(28.25), height=Cm(1.03))
    # 获取文本框文字对象
    tf = textBox.text_frame
    # 文本框样式调整
    #tf.margin_bottom = Cm(0.1)  # 下边距
    #tf.margin_left = 0  # 左边距
    tf.vertical_anchor = MSO_VERTICAL_ANCHOR.BOTTOM  # 垂直方式：底端对齐
    tf.word_wrap = True  # 文本框的文字自动对齐
    # 设置内容
    tf.paragraphs[0].text = content
    # 字体样式调整
    tf.paragraphs[0].alignment = PP_ALIGN.LEFT  # 对齐方式
    tf.paragraphs[0].font.name = '华文楷体'  # 字体名称
    tf.paragraphs[0].font.bold = True  # 是否加粗
    #tf.paragraphs[0].font.italic = True  # 是否斜体
    #tf.paragraphs[0].font.color.rgb = RGBColor(255, 0, 0)  # 字体颜色
    tf.paragraphs[0].font.size = Pt(20)  # 字体大小
    '''
    title = slide.placeholders[0]
    title.text=content
    #title.font.bold=True
    #title.font.size=Pt(20)
    tf = title.text_frame
    tf.paragraphs[0].font.name = '华文楷体'
    #tf.paragraphs[0].font.size = Pt(20)
    '''
def multi_invest(db):
    length = len(db)
    k = 0
    for i in range(0,length):
        invest = re.split(',|、|\n|，', db.loc[i,'投资机构(总)'])
        for j  in range(1,len(invest)):       
            db = db.append(db.iloc[i],ignore_index=True)
            db.loc[length+k,'投资机构(总)'] = invest[j]
            k = k+1
        db.loc[i,'投资机构(总)'] = invest[0]
    return db 

def sort_fuzzy_amount(db):
    m_100 = ['数亿','逾亿','超亿','过亿','上亿','亿级','亿元','近亿']
    m_10 = ['数千万','超千万','逾千万','千万级','近千万']
    m = ['数百万','上百万','超百万','逾百万','近百万']
    th_10 = ['数十万','上十万','超十万','逾十万','近十万']
    db_num = db[db['数值类型']=='数值']
    db_text = db[db['数值类型']=='文本']
    db_disclosure = db[db['数值类型']=='未披露']
    db_num = db_num.sort_values(['投资模糊金额(数值加金额)'],ascending=False)
    db_num_100m = db_num[db_num['投资模糊金额(数值加金额)']>=1]
    db_num_10m = db_num[(db_num['投资模糊金额(数值加金额)']>=0.1) & (db_num['投资模糊金额(数值加金额)']<1)]
    db_num_m = db_num[(db_num['投资模糊金额(数值加金额)']>=0.01) & (db_num['投资模糊金额(数值加金额)']<0.1)]
    db_num_th10 = db_num[(db_num['投资模糊金额(数值加金额)']>=0) & (db_num['投资模糊金额(数值加金额)']<0.01)]

    db_text_100m = pd.DataFrame()
    db_text_10m = pd.DataFrame()
    db_text_m = pd.DataFrame()
    db_text_th10 = pd.DataFrame()
    #亿元
    for i in m_100:
        db_text_100m = db_text_100m.append(db_text[db_text['投资模糊金额(数值加金额)'].str.contains(i)].sort_values('投资模糊金额(数值加金额)',ascending=False))
    db_text_100m_extra = db_text[db_text['投资模糊金额(数值加金额)'].str.contains('亿')].append(db_text_100m).drop_duplicates(keep=False)
    db_text_100m = db_text_100m.append(db_text_100m_extra)
    db_num_100m = db_num_100m.append(db_text_100m)
    #千万
    for i in m_10:
        db_text_10m = db_text_10m.append(db_text[db_text['投资模糊金额(数值加金额)'].str.contains(i)].sort_values('投资模糊金额(数值加金额)',ascending=False))
    db_text_10m_extra = db_text[db_text['投资模糊金额(数值加金额)'].str.contains('千万')].append(db_text_10m).drop_duplicates(keep=False)
    db_text_10m = db_text_10m.append(db_text_10m_extra)
    db_num_10m = db_num_10m.append(db_text_10m)
    #百万
    for i in m:
        db_text_m = db_text_m.append(db_text[db_text['投资模糊金额(数值加金额)'].str.contains(i)].sort_values('投资模糊金额(数值加金额)',ascending=False))
    db_text_m_extra = db_text[db_text['投资模糊金额(数值加金额)'].str.contains('百万')].append(db_text_m).drop_duplicates(keep=False)
    db_text_m = db_text_m.append(db_text_m_extra)
    db_num_m = db_num_m.append(db_text_m)
    #十万
    for i in th_10:
        db_text_th10 = db_text_th10.append(db_text[db_text['投资模糊金额(数值加金额)'].str.contains(i)].sort_values('投资模糊金额(数值加金额)',ascending=False))
    db_text_th10_extra = db_text[db_text['投资模糊金额(数值加金额)'].str.contains('十万')].append(db_text_th10).drop_duplicates(keep=False)
    db_text_th10 = db_text_th10.append(db_text_th10_extra)
    db_num_th10 = db_num_th10.append(db_text_th10)
    db = pd.concat([db_num_100m,db_num_10m,db_num_m,db_num_th10,db_disclosure],sort=False)
    db = db.drop_duplicates()
    db.to_excel(r'D:\Desktop\test.xlsx')
    return db

def re_expand(db,area):

    db1 = db.drop_duplicates()  # 按投资机构事件名称和一级赛道去重
    db1_new = db1[~db1['轮次'].isin(['IPO上市','私有化','并购','定向增发','股权转让'])]

    db2_new = get_indus_table_data(db1_new)
    # print(len(db2_new[(db2_new['标签（一级）']=='企业服务') & (db2_new['标签（二级）']=='软件')]))
#     first_industry_list = ['企业服务','交易平台','智能制造','消费','非关注领域']
    first_industry_list = ['企业服务','交易平台','智能制造','消费']
    for first_indus in first_industry_list:
        second_industry_list = list(set(db2_new[db2_new['标签（一级）'] == first_indus]['标签（二级）']))
        if first_indus == '企业服务': second_industry_list = ['软件','BPO和解决方案','医疗服务','物流服务']
        if first_indus == '交易平台': second_industry_list = ['Marketplace','柔性供应链','To B供应平台','To C供应平台']
        if first_indus == '智能制造': second_industry_list = ['优质供给','机器人','物联网']
        if first_indus == '消费': second_industry_list = ['品牌','零售','流量平台','其他（服务）']
        for second_indus in second_industry_list:
            content = str(first_indus) + '-'+str(second_indus)+'再展开'
            db3_new = db2_new[(db2_new['标签（一级）']==first_indus) & (db2_new['标签（二级）']==second_indus)]
            if first_indus == '消费':
                indu = first_indus                
            else:
                indu = second_indus
            print(indu,len(db3_new))
            db3_new = check_whitelist(db3_new,area,indu)
            data1 = db3_new[db3_new['投资金额(亿人民币)'] != 0 ]#不是空值
            data2 = db3_new[db3_new['投资金额(亿人民币)'] == 0 ]#空值
            # data1 = db3_new[db3_new['数值类型'] != '未披露' ]#不是空值
            # data2 = db3_new[db3_new['数值类型'] == '未披露' ]#空值
            list_custom_new = ['战略融资','股权融资','Pre-IPO','F轮','E轮','D+轮','D轮','C++轮','C+轮','C轮','B++轮','B+轮','B轮','Pre-B轮', 'A++轮','A+轮','Pre-A+轮','A2轮','A轮','Pre-A轮','天使轮','种子轮']
            data2['轮次'] = data2['轮次'].astype('category').cat.set_categories(list_custom_new)
            data1 = data1.sort_values('投资金额(亿人民币)', ascending=False)
            # data1 = sort_fuzzy_amount(data1)
            data2 = data2.sort_values('轮次', ascending=True)
            db3_new = data1.append(data2)
            db3_new = db3_new.reset_index()
            print(indu,len(db3_new))
            get_indu_reexpand_slide(db3_new,content,indu,area)
            
def check_whitelist(db,area,indu):
    if area == '中国' and indu == '物流服务':#如果改项目是中国，且非医疗机构
        white_list = pd.read_excel(path_to_excel, sheet_name='物流服务白名单')  # 导入白名单机构list
    elif area == '中国' and  indu == '医疗服务':#如果改项目是中国，且是医疗机构
        white_list = pd.read_excel(path_to_excel, sheet_name='医疗机构白名单')
    elif area == '中国' and indu == '消费':
        white_list = pd.read_excel(path_to_excel, sheet_name='消费组白名单')
    elif area == '国外':
        white_list = pd.read_excel(path_to_excel, sheet_name='海外白名单投资机构')  # 导入白名单机构list
    else:
        white_list = pd.read_excel(path_to_excel, sheet_name='国内白名单投资机构')
    whitelist = list(set(white_list['白名单投资机构']))
    ##跟"投资机构"列匹配code,得到白名单机构的投资事件db3
    db['是否白名单投资'] = 0
    for i in whitelist:
        for j in db.index:
            if pd.isnull(db['投资机构(总)'][j]) == False:
                if i in db['投资机构(总)'][j]:
                    db['是否白名单投资'][j] = 1
    return db

def check_white(jigou,area,indu):
    if area == '中国' and indu == '物流服务':#如果改项目是中国，且非医疗机构
        white_list = pd.read_excel(path_to_excel, sheet_name='物流服务白名单')  # 导入白名单机构list
    elif area == '中国' and  indu == '医疗服务':#如果改项目是中国，且是医疗机构
        white_list = pd.read_excel(path_to_excel, sheet_name='医疗机构白名单')
    elif area == '中国' and indu == '消费':
        white_list = pd.read_excel(path_to_excel, sheet_name='消费组白名单')
    elif area == '国外':
        white_list = pd.read_excel(path_to_excel, sheet_name='海外白名单投资机构')  # 导入白名单机构list
    else:
        white_list = pd.read_excel(path_to_excel, sheet_name='国内白名单投资机构') 
    whitelist = list(set(white_list['白名单投资机构']))
    abnornal_whitelist = pd.read_excel(path_to_excel, sheet_name='歧义词')
    abnornal_whitelist = list(abnornal_whitelist['歧义机构名'])
    result = False
    flag = 1
    for i in whitelist:
        if i in jigou:
            for j in abnornal_whitelist:
                if jigou == j:
                    flag = 0
                    break
            if flag == 1:
                result = True
                break
    return result

'''
height_space: 上下边界空白
width_space：左右边界空白
font_size：字号
width：单元格宽度
'''
def cal_height(height_space,width_space,font_size,width,text):
    text_len = len(text)
    num_per_line = (width-2*width_space)//0.035146#计算出一行可以放几个字符
    if text_len % num_per_line ==0:
        num_line = text_len // num_per_line
    else:
        num_line = (text_len // num_per_line) +1
    height = 2*height_space+num_line*font_size*0.035146
    return height
def date_to_ymd(x):
    return x.strftime('%Y-%m-%d')
#将所有文本金额转成数值金额
def transfer_amount(db):
    amount = list(set(db['投资金额(文本)']))
    df = pd.DataFrame()
    exchange = 1
    for i in amount:
        string = i
        if string == '未披露':
            df = df.append([[string,0]])
        else:
            if string.find('人民币') != -1:
                exchange = 1
            elif string.find('美元') != -1:
                exchange = 6.45
            elif string.find('欧元') != -1:
                exchange = 7.81
            elif string.find('英镑') != -1:
                exchange = 8.84
            elif string.find('瑞士法郎') != -1:
                exchange = 7.25
            elif string.find('卢币') != -1:
                exchange = 0.0885
            elif string.find('港币') != -1:
                exchange = 0.88
            elif string.find('港元') != -1:
                exchange = 0.88
            #确认数值
            num_list = re.findall(r"\d+\.?\d*",string)
            if len(num_list) == 0:
                flag = 0
                several = ['数千万','数百万','数十万','数亿']
                more = ['逾亿','超亿','上亿','过亿','超千万','逾千万','上百万','超百万','逾百万']
                equal = ['亿元','千万级','百万']
                less = ['近亿','近千万','近百万']
                for i in several:
                    if i in string:
                        num = 5
                        flag = 1
                        break
                if flag == 0:
                    for i in more:
                        if i in string:
                            num = 1.2
                            flag = 1
                            break
                if flag == 0:
                    for i in equal:
                        if i in string:
                            num = 1
                            flag = 1
                            break
                if flag == 0 :
                    for i in less:
                        if i in string:
                            num=0.8
                            break
            else:
                num = float(num_list[0])
            #确认单位换算
            if string.find('亿') != -1:
                num = num*10000
            elif string.find('千万') != -1:
                num = num*1000
            elif string.find('百万') != -1:
                num = num*100
            elif string.find('十万') != -1:
                num = num*10
            elif string.find('万') != -1:
                num = num*1
            df = df.append([[string,num*exchange]])
    df = df.rename(columns={0:'投资金额(文本)',1:'投资模糊金额(万)'})
    db = pd.merge(db,df,on = '投资金额(文本)',how='left')
    return db
#只转换确定数值的的，模糊的保留本文
def transfer_amount1(db):
    amount = list(set(db['投资金额(文本)']))
    df = pd.DataFrame()
    exchange = 1
    for i in amount:
        string = i
        #确认数值
        num_list = re.findall(r"\d+\.?\d*",string)
        if string == '未披露':
            df = df.append([[string,'/','未披露']])
        #如果有确切数值
        elif len(num_list) > 0:
            if string.find('人民币') != -1:
                exchange = 1
            elif string.find('美元') != -1:
                exchange = 6.45
            elif string.find('欧元') != -1:
                exchange = 7.81
            elif string.find('英镑') != -1:
                exchange = 8.84
            elif string.find('瑞士法郎') != -1:
                exchange = 7.25
            elif string.find('卢币') != -1:
                exchange = 0.0885
            elif string.find('港币') != -1:
                exchange = 0.88
            elif string.find('港元') != -1:
                exchange = 0.88
            num = float(num_list[0])
            #确认单位换算
            if string.find('亿') != -1:
                num = num*10000
            elif string.find('千万') != -1:
                num = num*1000
            elif string.find('百万') != -1:
                num = num*100
            elif string.find('十万') != -1:
                num = num*10
            elif string.find('万') != -1:
                num = num*1
            df = df.append([[string,round(num*exchange/10000,1),'数值']])
        #如果没有确切数值
        else:
            nums = ['数千万','数百万','数十万','数亿','逾亿','超亿','近亿','上亿','过亿','亿元','千万级','超千万','逾千万','近千万','上百万','超百万','逾百万','百万','近百万']
            flag1 = 0
            flag2 = 0
            for i in nums:
                if i in string:
                    num = i
                    flag1 = 1
                    break
            currencies = ['人民币','美元','欧元','英镑','瑞士法郎','卢币','港币','港元']
            for i in currencies:
                if i in string:
                    currency = i
                    flag2 = 1
                    break
            if flag1 == 0: num=''
            if flag2 == 0: currency=''
            df = df.append([[string,num+currency,'文本']])
    df = df.rename(columns={0:'投资金额(文本)',1:'投资模糊金额(数值加金额)',2:'数值类型'})
    db = pd.merge(db,df,on = '投资金额(文本)',how='left')
    return db
#将模糊投资金额列为空的用投资金额文本补齐
def get_all_amount(db):
    #将投资金额转换为数值
    db1 = db[pd.isnull(db['投资模糊金额(万)'])==True]
    db2 = db[pd.isnull(db['投资模糊金额(万)']) == False]
    db1 = transfer_amount(db1)
    db1['投资模糊金额(万)'] = db1['投资金额(数值)']
    db1 = db1.drop(columns='投资金额(数值)')
    db = db2.append(db1)
    db = db.reset_index(drop=True)
    return db
#判断是否首次进入/追加投资
def check_whether_first(db,total_data):
    projects = set(db['项目'])
    invest = list(set(db['投资机构(总)']))[0]
    db['日期'] = db['日期'].astype('str')
    final_data= pd.DataFrame()
    for project in projects:
        data1 = db[db['项目'] == project]
        # if data1.empty:
        data1 = data1.reset_index(drop=True)
        data2 = total_data[(total_data['项目']==project) & (total_data['投资机构(总)'].str.contains(invest))]
        if data2.empty:
            print(invest,project)
            final_data = final_data.append(data1)
            continue
        data2 = data2.sort_values(by=['日期'])
        data2 = data2.reset_index(drop=True)
        for i in range(len(data1)):
            try:
                if data1.loc[i,'日期'] == data2.loc[0,'日期']:
                    data1.loc[i:,'首次进入/追加投资'] = '首次进入'
                else:
                    data1.loc[i:,'首次进入/追加投资'] = '追加投资'
            except:
                pass
        final_data = final_data.append(data1)
    return  final_data

print('OK')

OK


In [4]:
#处理文本形式的金额
origin_data = pd.read_excel(r'D:\Desktop\钟鼎资本\代码\全市场跟踪\6月项目-2.xlsx')
origin_data['投资金额(文本)'] = origin_data['投资金额(文本)'].astype(str)
origin_data = transfer_amount(origin_data)
origin_data = transfer_amount1(origin_data)
origin_data.to_excel(r'D:\Desktop\钟鼎资本\代码\全市场跟踪\6月项目-2.xlsx',index=False)

In [8]:
#生成白名单列
origin_data = pd.read_excel(r'D:\Desktop\钟鼎资本\代码\全市场跟踪\6月项目-2.xlsx')
origin_data['投资机构(总)'] = origin_data['投资机构(总)'].astype(str)
origin_data['是否白名单'] = ''
for i in origin_data.index:
    if origin_data.loc[i,'标签（一级）'] == '消费':
        indu = '消费'
    else :
        inde = origin_data.loc[i,'标签（二级）']
    if check_white(origin_data.loc[i,'投资机构(总)'],'中国',inde):
        origin_data.loc[i,'是否白名单'] = 1
    else:
        origin_data.loc[i,'是否白名单'] = 0
origin_data.to_excel(r'D:\Desktop\钟鼎资本\代码\全市场跟踪\6月项目-2.xlsx',index=False)

In [49]:
#匹配A、B轮
origin_data = pd.read_excel(r'D:\Desktop\钟鼎资本\代码\全市场跟踪\6月项目-2.xlsx')
origin_data = origin_data.drop_duplicates()
total_data = pd.read_excel(path_to_project)
white_com = list(set(pd.read_excel(path_to_excel,sheet_name='国内白名单投资机构')['白名单投资机构']))
#如果出现A，B轮有多次融资的情况，保留最近期的轮次
total_data = total_data.sort_values(by='日期',ascending=False)
total_data = total_data.drop_duplicates()
projects = list(set(origin_data['项目']))
turns = ['天使轮','种子轮']
A = pd.DataFrame()
B = pd.DataFrame()
non_word = ['领投','跟投','等跟投']
for project in projects:
    df1 = total_data[total_data['项目'] == project]
    end_time = df1[df1['轮次'].str.contains('A')]['日期'].max()
    df2 = df1[df1['日期']<=end_time]
    if not df2.empty:
        A_invest = ','.join(list(df2['投资机构(总)']))
        #处理重复的白名单投资机构
        A_invest = re.split(',|、|\n|，', A_invest)
        A_new_invest = []
        #处理白名单机构
        for i in white_com:
            flag = 0
            for j in A_invest:
                if (i in j) and (flag==0) :
                    A_new_invest.append(j)
                    flag = 1
        #添加非白名单机构
        for j in A_invest:
            flag = 0
            for i in white_com:
                if i in j :
                    flag = 1
            if (flag==0) and (j!='未披露'):
                for word in non_word:
                    if j.find(word)!=-1:
                        j = j[0:j.find(word)]
                A_new_invest.append(j)
        A_new_invest = list(set(A_new_invest))#去除非白名单里重复的
        df2 = df2.reset_index(drop=True)
        df2.loc[0,'投资机构(总)'] = ','.join(A_new_invest)
        df2 = df2.drop_duplicates('项目',keep='first')
        A = A.append(df2)
    #处理泛B轮
    df3 = df1[df1['轮次'].str.contains('B')]
    df3 = df3.reset_index(drop=True)
    if not df3.empty:
        B_invest = ','.join(list(df3['投资机构(总)']))
        #处理重复的白名单投资机构
        B_invest = re.split(',|、|\n|，', B_invest)
        B_new_invest = []
        #处理白名单机构
        for i in white_com:
            flag = 0
            for j in B_invest:
                if (i in j) and (flag==0) :
                    B_new_invest.append(j)
                    flag = 1
        #添加非白名单机构
        for j in B_invest:
            flag = 0
            for i in white_com:
                if i in j :
                    flag = 1
            if (flag==0) and (j!='未披露'):
                for word in non_word:
                    if j.find(word)!=-1:
                        j = j[0:j.find(word)]
                B_new_invest.append(j)

        B_new_invest = list(set(B_new_invest))#去除非白名单里重复的
        df3=df3.reset_index(drop=True)
        df3.loc[0,'投资机构(总)'] = ','.join(B_new_invest)
        df3 = df3.drop_duplicates('项目',keep='first')
        B = B.append(df3)

# A = total_data[total_data['轮次']=='A轮'][['项目','投资机构(总)']]
A = A[['项目','投资机构(总)']]
B = B[['项目','投资机构(总)']]

# A = A.drop_duplicates(subset=['项目'])
A.rename(columns={'投资机构(总)':'A轮及以前投资机构'},inplace=True)
# B = total_data[total_data['轮次'].str.contains('B')][['项目','投资机构(总)']]
# B = B.drop_duplicates(subset=['项目'])
B.rename(columns={'投资机构(总)':'B轮投资机构'},inplace=True)
origin_data = origin_data.merge(A,how='left',on='项目')
origin_data = origin_data.merge(B,how='left',on='项目')
origin_data['A轮及以前投资机构'].fillna('/',inplace=True)
origin_data['B轮投资机构'].fillna('/',inplace=True)
origin_data.to_excel(r'D:\Desktop\钟鼎资本\代码\全市场跟踪\6月项目-3.xlsx',index=False)

#%


In [ ]:
# 判断是不是当前轮
origin_data = pd.read_excel(r'D:\Desktop\钟鼎资本\代码\全市场跟踪\6月项目-3.xlsx')
origin_data = origin_data.drop_duplicates()
origin_data['投资机构(总)'] = origin_data['投资机构(总)'].astype(str)
turns = ['A轮','Pre-A轮','天使轮','种子轮']
for i in range(len(origin_data)):
    if origin_data.loc[i,'轮次'] in turns:
        origin_data.loc[i,'A轮及以前投资机构'] = '当前轮'
    if origin_data.loc[i,'轮次'] == 'B轮':
        origin_data.loc[i,'B轮投资机构'] = '当前轮'
origin_data.to_excel(r'D:\Desktop\钟鼎资本\代码\全市场跟踪\6月项目-3.xlsx',index=False)
# for i in range(0,len(origin_data)):
#     A = re.split(',|、|\n|，', str(origin_data.loc[i,'A轮投资机构']))
#     B = re.split(',|、|\n|，', str(origin_data.loc[i,'B轮投资机构']))
#     Total = re.split(',|、|\n|，', origin_data.loc[i,'投资机构(总)'])
#     a_flag = 0
#     b_flag = 0
#     for a in A:
#         for total in Total:
#             if a.find(total)!=-1 or total.find(a)!=-1:
#                 a_flag = a_flag+1
#                 continue
#     for b in B:
#         for total in Total:
#             if b.find(total)!=-1 or total.find(b)!=-1:
#                 b_flag = b_flag+1
#                 continue
#     if a_flag == len(A) and a_flag == len(Total):
#         origin_data.loc[i,'A轮投资机构'] = '当前轮'
#     if b_flag == len(B) and b_flag == len(Total):
#         origin_data.loc[i,'B轮投资机构'] = '当前轮'


In [51]:
# 赛道再展开
origin_data = pd.read_excel(r'D:\Desktop\钟鼎资本\代码\全市场跟踪\6月项目-3.xlsx')
origin_data['标签'] = origin_data['标签（一级）']+'/'+origin_data['标签（二级）']
origin_data['A轮及以前投资机构'] = origin_data['A轮及以前投资机构'].fillna(value='/')
origin_data['B轮投资机构'] = origin_data['B轮投资机构'].fillna(value='/')
prs = Presentation(path_to_template)
re_expand(origin_data,'中国')
prs.save(r'D:\Desktop\赛道再展开.pptx')


软件 94
软件 94
BPO和解决方案 42
BPO和解决方案 42
医疗服务 70
医疗服务 70
物流服务 13
物流服务 13
Marketplace 17
Marketplace 17
柔性供应链 0
柔性供应链 0
To B供应平台 6
To B供应平台 6
To C供应平台 11
To C供应平台 11
优质供给 84
优质供给 84
机器人 8
机器人 8
物联网 20
物联网 20
消费 64
消费 64
消费 3
消费 3
消费 9
消费 9
消费 25
消费 25


C:\Users\ZSJ\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ZSJ\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:581: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\ZSJ\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:548: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [23]:
# 分机构展开
prs = Presentation(path_to_template)
origin_data = pd.read_excel(r'D:\Desktop\钟鼎资本\代码\全市场跟踪\6月项目-3.xlsx')
origin_data['投资机构(总)'] = origin_data['投资机构(总)'].astype(str)
start_time = '2021-06-01'
end_time = '2021-06-30'
origin_data['钟鼎标签'] = origin_data['标签（一级）']+'/'+origin_data['标签（二级）']
origin_data['A轮及以前投资机构'] = origin_data['A轮及以前投资机构'].fillna(value='/')
origin_data['B轮投资机构'] = origin_data['B轮投资机构'].fillna(value='/')
origin_data['首次进入/追加投资'] = ''
origin_data['钟鼎标签'] = origin_data['钟鼎标签'].fillna(value='非关注领域')

time_period = "6.1-6.30"
cover('中国一级市场白名单机构投资事件汇总')
white_expand(origin_data, '中国',time_period)
prs.save('D:\Desktop\分机构展开.pptx')


C:\Users\ZSJ\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:428: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
E:\python\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\ZSJ\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [5]:
#IPO top20展开
prs = Presentation(path_to_template)
origin_data = pd.read_excel('D:\Desktop\钟鼎资本\代码\IPO数据\全部A股IPO.xlsx')
start_time = '2021-06-01'
end_time = '2021-06-30'
origin_data['交易所'] = origin_data['交易所'].replace('上海证券交易所','上交所')
origin_data['交易所'] = origin_data['交易所'].replace('深圳证券交易所','深交所')
origin_data = origin_data[(origin_data['上市日期']>=start_time) & (origin_data['上市日期']<=end_time)]
origin_data = origin_data.sort_values('收盘日总市值',ascending=False)
origin_data = origin_data.iloc[0:20]
origin_data.to_excel('D:\Desktop\ipo_top20.xlsx',index=False)
content = '中国新IPO公司Top 20（按当前市值排序）'
def get_ipo20_slide(db,content):
    #获得投资金额(亿人民币)这一列
    df2 = db[['证券简称','钟鼎标签','公司简介','城市','投资机构','交易所','上市日总市值','收盘日总市值']].values

    num_per_page = 5
    pages_f = lambda x: (x // num_per_page) + 1 if x % num_per_page > 0 else x // num_per_page
    pages = pages_f(len(df2))
    df2 = np.insert(df2, 0, values=np.array([['项目','钟鼎标签','公司简介','城市','投资机构','交易所','IPO市值（亿¥）','当前市值（亿¥）注：括号内为市值增长率']]), axis=0)
    for i in range(0, pages):
        slide = prs.slides.add_slide(prs.slide_layouts[1])
        content1 = content+'（'+str(i+1)+'/'+str(pages)+'）'
        add_title(slide, content1)
        # 设置表格位置和大小
        left, top, width, height = Cm(0.1), Cm(2.7), Cm(24.46), Cm(14.97)
        rows, cols = min(len(df2),num_per_page+1), df2.shape[1]
        # 表格行列数，和大小
        shape = slide.shapes.add_table(rows, cols, left, top, width, height)
        table = shape.table
        table.columns[0].width = Cm(1.7)
        table.columns[1].width = Cm(2.5)
        table.columns[2].width = Cm(7)
        table.columns[3].width = Cm(1.5)
        table.columns[4].width = Cm(4)
        table.columns[5].width = Cm(3)
        table.columns[6].width = Cm(2)
        table.columns[7].width = Cm(3.5)
        # table.columns[8].width = Cm(3.3)
#         table.columns[9].width = Cm(3)

        # 赋值，调格式
        for row in range(rows):
            table.rows[row].height = Cm(0.92)
            for col in range(cols):
                if row == 0:
                    table.cell(row,col).text = str(df2[0][col])
                else:
                    if type(df2[1][col]) == type(12.5):#如果是金额列，要保留以为小数
                        table.cell(row, col).text = format(df2[1][col],'.1f')
                    elif col == 3:#如果是城市列，要取出“市”
                        if str(df2[1][col]).find('市') != -1:
                            table.cell(row, col).text = str(df2[1][col])[0:str(df2[1][col]).find('市')]
                        else:
                            table.cell(row, col).text = str(df2[1][col])
                    else:#其他列
                        if pd.isnull(df2[1][col]) == False:
                            table.cell(row, col).text = str(df2[1][col])
                        else:
                            table.cell(row, col).text = '/'

                # 设置文字大小
                table.cell(row, col).text_frame.paragraphs[0].font.size = Pt(11)
                # 设置字体
                table.cell(row, col).text_frame.paragraphs[0].font.name = '华文楷体'
                # table.cell(row, col).text_frame.paragraphs[0].font.name = 'Corbel'
                # 设置文字颜色
                # table.cell(row, col).text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
                # 设置文字左右对齐
                table.cell(row, col).text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                # 设置文字上下对齐
                table.cell(row, col).vertical_anchor = MSO_ANCHOR.MIDDLE
                # 设置背景为填充
                # table.cell(row, col).fill.solid()

            if row > 0:
                df2 = np.delete(df2, 1, 0)#删除第一行
get_ipo20_slide(origin_data,content)
prs.save('D:\Desktop\ipo_top20.pptx')



In [26]:
'''
粉表
'''
def check_non_top20(db):

    db1 = db.sort_values(by = '投资模糊金额(万)',ascending=False)
    db1 = db1.reset_index(drop=True)
    industry = list(set(db1['标签（一级）']))#重复元素在set中自动过滤
    db2 = pd.DataFrame()
    for j in industry:
        db_j = db1[db1['标签（一级）'] == j]
        #db_j['是否不属于top20'] = 1
        db_j = db_j.reset_index(drop=True)
        db_j_20 = db_j.loc[19:]
        #for i in db_j.index:
        #    if i<20:
        #        temp = db_j['是否不属于top20'][i]
        db2 = db2.append(db_j_20)
    top20_list = list(set(db2['项目']))
    return top20_list

#生成数据透视表并排序函数
def white_invest_vlookup_table(db3):
#     white_invest1 = db3.pivot_table(values='名称', index=['一级赛道', '二级赛道'], columns='白名单投资机构', aggfunc="count",
#                                     margins=True)
    white_invest1 = db3.pivot_table(values='项目', index=['标签（一级）', '标签（二级）'], columns='白名单投资机构', aggfunc="count",
                                    margins=True)
#     white_invest2 = db3.pivot_table(values='名称', index=['一级赛道'], columns='白名单投资机构', aggfunc="count", margins=True)
    white_invest2 = db3.pivot_table(values='项目', index=['标签（一级）'], columns='白名单投资机构', aggfunc="count", margins=True)
    white_invest1 = white_invest1.drop(index='All')
    white_invest1.reset_index(inplace=True)
    white_invest2.reset_index(inplace=True)
    white_invest3 = white_invest1.append(white_invest2)
    # 对white_invest3排序
    #修改一级赛道和二级赛道的名称
    white_invest3 = white_invest3.reset_index(drop=True)
    white_invest3['标签（二级）'] = white_invest3['标签（二级）'].fillna(value='subtotal')
    white_invest3['标签（二级）'][white_invest3.index[white_invest3['标签（一级）'] == 'All']] = 'All'
    #对投资机构排序（按照总投资数量降序排序）
    white_invest3 = white_invest3.set_index(['标签（一级）', '标签（二级）'])
    white_invest4 = white_invest3.sort_values(by=[('All', 'All')], ascending=False, axis=1)
    white_ranked = white_invest4
    # 将all列移动到最后
    all = white_invest4.pop('All')
    white_invest4.insert(len(white_invest4.columns), 'All', all)
    #将一级赛道和二级赛道按自定义顺序排序
    sorted1 = ['企业服务', '交易平台', '智能制造', '消费', '非关注领域', ' ','All']
    sorted2 = ['subtotal', '软件','BPO和解决方案', '医疗服务', '物流服务','Marketplace', '柔性供应链', 'To B供应平台','To C供应平台',  '优质供给', '机器人',  '物联网', '品牌', '零售', '流量平台', '其他（服务）',  ' ', 'All']
    white_invest5 = white_invest4.reset_index()
    white_invest5['标签（一级）'] = white_invest5['标签（一级）'].astype('category').cat.set_categories(sorted1)
    white_invest5['标签（二级）'] = white_invest5['标签（二级）'].astype('category').cat.set_categories(sorted2)
    df_sortes2 = white_invest5.sort_values(by=['标签（一级）', '标签（二级）'], ascending=True, na_position='last')  # 将赛道排序
    white_table = df_sortes2.T
    return white_table

def part1_pink(db,area):
    #是否属于top20公司
    top20_list = check_non_top20(db)

    #4-白名单投资机构数据筛选
    db3 = match_invest_with_whitelist(db,area)
    db3 = db3.reset_index(drop=True)

    #5-生成数据透视表并排序
    white_table = white_invest_vlookup_table(db3)
    white_table = merge_industry(white_table)
    content = '各白名单机构投资赛道分布情况（排序：按各机构总投资数）'
    get_pink_table_slide(white_table,content)

def get_pink_table_slide(white_table,content):
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    #content=str(area)+'白名单机构投资事件分析'
    add_title(slide,content)
    #white_table = white_table.rename(columns={'All-All':'All'})
    white_table = white_table.reset_index()
    df2 = white_table.values
    df2 = df2[0:24]
    df2[0][0] = '白名单投资机构'
    # 设置表格位置和大小
    left, top, width, height = Cm(0.45), Cm(2.64), Cm(24.51), Cm(15.63)
    rows,cols = len(df2),len(df2[0])
    # 表格行列数，和大小
    shape = slide.shapes.add_table(rows, cols, left, top, width, height)
    table = shape.table
    #赋值，调格式
    col_list = []
    all_col=-1
    for row in range(rows):
        table.rows[row].height= Cm(0.43)
        for col in range(cols):
            _set_cell_border(table.cell(row, col))
            if (type(df2[row][col]) == type(12.3)):
                if (df2[row][col]>0)==False:
                    table.cell(row, col).text = ""
                else:
                    table.cell(row, col).text = format(df2[row][col],'.0f')
            elif df2[row][col] == 'All-All':
                table.cell(row, col).text = 'All'
            elif df2[row][col] == '2B服务-解决方案（运维系统+服务）':
                table.cell(row, col).text = '2B服务-解决方案'
            else:
                table.cell(row, col).text = str(df2[row][col])

            # 设置文字大小
            table.cell(row, col).text_frame.paragraphs[0].font.size = Pt(8)
            # 设置字体
            table.cell(row, col).text_frame.paragraphs[0].font.name = '华文楷体'
            # 设置文字左右对齐
            table.cell(row, col).text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
            # 设置文字上下对齐
            table.cell(row, col).vertical_anchor = MSO_ANCHOR.MIDDLE
            # 设置背景为填充
            table.cell(row, col).fill.solid()
            table.cell(row, col).text_frame.paragraphs[0].font.color.rgb = RGBColor(0,0,0)
            table.cell(row, col).fill.fore_color.rgb = RGBColor(255,255,255)
            #title加粗&设置文字和背景颜色
            #if (row ==0) | (col==0):
            #    table.cell(row, col).text_frame.paragraphs[0].font.bold =True
            #汇总列加粗
            if (table.cell(row, col).text.find('All')>=0) & (row == 0):
                all_col = col
            if col == all_col:
                table.cell(row, col).text_frame.paragraphs[0].font.bold = True
                table.cell(row, col).fill.fore_color.rgb = RGBColor(246, 164, 160)
            else:
                table.cell(row, col).text_frame.paragraphs[0].font.bold = False
            #subtotal列加颜色
            if table.cell(row, col).text.find('subtotal')>=0:
                col_list.append(col)
            if (col in col_list)==True:
                table.cell(row, col).fill.fore_color.rgb = RGBColor(242, 220, 219)
            else:
                table.cell(row, col).text_frame.paragraphs[0].font.color.rgb = RGBColor(0,0,0)

def merge_industry(white_table):
    # white_table.loc['赛道'] = white_table.loc['标签（一级）']+'-'+white_table.loc['标签（二级）']
    white_table.loc['赛道'] = white_table.loc['标签（二级）']
    white_table2= white_table.T
    temp = white_table2.pop('赛道')
    white_table2.insert(0, '赛道', temp)
    white_table2 = white_table2.drop(columns=['标签（一级）','标签（二级）'])
    white_table3 = white_table2.T
    return white_table3
def _set_cell_border(cell, border_color="000000", border_width='12700'):
       tc = cell._tc
       tcPr = tc.get_or_add_tcPr()
       for lines in ['a:lnL','a:lnR','a:lnT','a:lnB']:
           ln = SubElement(tcPr, lines, w=border_width, cap='flat', cmpd='sng', algn='ctr')
           solidFill = SubElement(ln, 'a:solidFill')
           srgbClr = SubElement(solidFill, 'a:srgbClr', val=border_color)
           prstDash = SubElement(ln, 'a:prstDash', val='solid')
           round_ = SubElement(ln, 'a:round')
           headEnd = SubElement(ln, 'a:headEnd', type='none', w='med', len='med')
           tailEnd = SubElement(ln, 'a:tailEnd', type='none', w='med', len='med')
#表格添加边框函数
def SubElement(parent, tagname, **kwargs):
           element = OxmlElement(tagname)
           element.attrib.update(kwargs)
           parent.append(element)
           return element
origin_data = pd.read_excel(r'D:\Desktop\钟鼎资本\代码\全市场跟踪\6月项目-3.xlsx')
prs = Presentation(path_to_template)
part1_pink(origin_data,'中国')
prs.save(r'D:\Desktop\粉表.pptx')

C:\Users\ZSJ\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:428: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
E:\python\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
E:\python\lib\site-packages\pandas\core\generic.py:3935: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
C:\Users\ZSJ\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value 

In [9]:
#投融资额Top20
def get_top20_project_slide(data,content,area):
    data = data[data['标签（一级）']!='非关注领域']
    data = get_indus_table_data(data)
    data = data[(data['轮次']=='种子轮')|(data['轮次']=='天使轮')|(data['轮次']=='Pre-A轮')|(data['轮次']=='A轮')]
    data = data.sort_values(by=['投资金额(亿人民币)'],ascending=False)
    data = data.iloc[0:20]
    df2 = data[['项目','钟鼎标签','业务','城市','轮次','投资金额(亿人民币)','投资机构(总)','A轮及以前投资机构','B轮投资机构','标签（一级）','标签（二级）']].values
    df2 = np.insert(df2, 0, values=np.array([['项目','钟鼎标签','业务','城市','轮次','投资金额(亿￥)','最新一轮投资机构','A轮及以前投资机构','B轮投资机构','标签（一级）','标签（二级）']]), axis=0)
    font_size = 11
    num_per_page = 5
    pages_f = lambda x: (x // num_per_page) + 1 if x % num_per_page > 0 else x // num_per_page
    pages = pages_f(len(df2)-1)
    # length = len(df2)
    # height = 0
    # pages = 0
    # max_height = []
    # num_per_page = []
    # flag = 0
    # col_index = [2,6,7,8]
    #找到需要几页，每页能容纳几个记录
    # for i in range(0,length):
    #     for j in col_index:
    #         if j == 6 or j == 2 or j==7 or j==8:
    #             max_height.append(cal_height(0.46541,0.269889,font_size,4,df2[i][j]))
    #         else:
    #             max_height.append(cal_height(0.46541,0.269889,font_size,2.712,df2[i][j]))
    #     max_height.sort()
    #     height = height+max_height[-1]
    #     if height <= 5.833:
    #         if i == length-1 :#如果全部记录长度连一页都不够
    #             if pages == 0:
    #                 pages = 1
    #                 num_per_page.append(length)
    #                 # print('188',str(i)+'行',height)
    #             else:#最后一页长度不满一页
    #                 pages = pages + 1
    #                 num_per_page.append(i-flag+1)
    #                 # print('192',str(i)+'行',height)
    #     else:
    #         # print('198',str(i)+'行',height)
    #         height = 0.92#0.128228为第一行的高度
    #         if i == length-1:
    #             num_per_page.append(i-flag)
    #             num_per_page.append(1)
    #             pages = pages+1
    #         else:
    #             num_per_page.append(i-flag)
    #             flag = i
    #         pages = pages + 1
    #         #除去表头，每页还能容纳几行,第一页是算上了表头，之后几页没算上
    # for j in range(1,len(num_per_page)):
    #     num_per_page[j] = num_per_page[j]+1

    for i in range(0, pages):
        slide = prs.slides.add_slide(prs.slide_layouts[1])
        #content = '白名单投资机构投资事件'
        content1 = content+'('+str(i+1)+'/'+str(pages)+')'
        add_title(slide, content1)
        # 设置表格位置和大小
        left, top, width, height = Cm(0.1), Cm(2.7), Cm(24.46), Cm(15.17)
        rows,cols = min(len(df2),num_per_page+1),df2.shape[1]-2

        # 表格行列数，和大小
        shape = slide.shapes.add_table(rows, cols, left, top, width, height)

#     num_per_page = 5
#     pages_f = lambda x: (x // num_per_page) + 1 if x % num_per_page > 0 else x // num_per_page
#     pages = pages_f(len(df2)-1)
#     area = '中国'
        table = shape.table
        table.columns[0].width = Cm(1.7)
        table.columns[1].width = Cm(2.5)
        table.columns[2].width = Cm(5)
        table.columns[3].width = Cm(1.5)
        table.columns[4].width = Cm(1.5)
        table.columns[5].width = Cm(2)
        table.columns[6].width = Cm(3.67)
        table.columns[7].width = Cm(3.67)
        table.columns[8].width = Cm(3.67)
        # 赋值，调格式
        for row in range(0,rows):
            table.rows[0].height = Cm(0.92)
            for col in range(cols):
                if row == 0:
                    table.cell(row,col).text = str(df2[0][col])
                else:
                    if df2[0][col] == '城市':#如果是城市列，要取出“市”
                        if str(df2[1][col]).find('市') != -1:
                            table.cell(row, col).text = str(df2[1][col])[0:str(df2[1][col]).find('市')]
                        else:
                            table.cell(row, col).text = str(df2[1][col])

                    elif df2[0][col] == '投资金额(亿￥)':#如果是投资金额列
                        if pd.isnull(df2[1][col]) == False:
                            if df2[1][col] == 0:
                                table.cell(row, col).text = '/'#如果为零，用斜杆替换
                            else:
                                table.cell(row, col).text = format(df2[1][col],'.1f')
                        else:
                            table.cell(row, col).text = '/'#如果空值，用斜杆替换

                    elif df2[0][col] == '最新一轮投资机构' or df2[0][col] == 'A轮及以前投资机构' or df2[0][col] == 'B轮投资机构':
                        if df2[1][9] == '消费': indu = df2[1][9]
                        else: indu = df2[1][10]
                        red_signal(df2[1][col],table,row,col,area,indu)

                    else:#其他列
                        if pd.isnull(df2[1][col]) == False:
                            table.cell(row, col).text = str(df2[1][col])
                        else:
                            table.cell(row, col).text = '/'
                # 设置文字大小
                table.cell(row, col).text_frame.paragraphs[0].font.size = Pt(font_size)
                # 设置字体
                table.cell(row, col).text_frame.paragraphs[0].font.name = '华文楷体'
                # table.cell(row, col).text_frame.paragraphs[0].font.name = 'Corbel'
                # 设置文字颜色
                # table.cell(row, col).text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
                # 设置文字左右对齐
                table.cell(row, col).text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                # 设置文字上下对齐
                table.cell(row, col).vertical_anchor = MSO_ANCHOR.MIDDLE
                # 设置背景为填充
                # table.cell(row, col).fill.solid()

            if row > 0:
                df2 = np.delete(df2, 1, 0)#删除第一行

origin_data = pd.read_excel(r'D:\Desktop\钟鼎资本\代码\全市场跟踪\6月项目-3.xlsx')
origin_data['钟鼎标签'] = origin_data['标签（一级）']+'/'+origin_data['标签（二级）']
prs = Presentation(r'D:\Desktop\钟鼎资本\代码\自动化PPT模版.pptx')
content = '中国一级市场热点项目投融资额Top20'
get_top20_project_slide(origin_data,content,'中国')
prs.save(r'D:\Desktop\投融资top20.pptx')

C:\Users\ZSJ\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
origin_data = pd.read_excel('D:\Desktop\钟鼎资本\代码\软件市场跟踪\所需文件\底库不去重.xlsx')
origin_data = origin_data[~origin_data['轮次'].isin(['IPO上市', '私有化', '并购', '定向增发', '股权转让'])]
origin_data = origin_data.drop_duplicates()
def add_square_title(slide,title):
    left, top, width, height = Cm(0.1), Cm(2.7), Cm(9.5), Cm(1)
    text_box = slide.shapes.add_textbox(left, top, width, height)
    tf = text_box.text_frame
    tf.text = title
    tf.vertical_anchor = MSO_ANCHOR.MIDDLE
    tf.paragraphs[0].alignment = PP_ALIGN.CENTER
    tf.paragraphs[0].font.name = '华文楷体'
    tf.paragraphs[0].font.size = Pt(14)
    tf.paragraphs[0].font.bold = True
    tf.paragraphs[0].font.color.rgb = RGBColor(255,255,255)
    fill = text_box.fill
    fill.solid()
    # 使用之前一定要导入RGBColor这个库
    fill.fore_color.rgb = RGBColor(85, 114, 197)
def get_software_expand_slide(db,content,title):
    font_size = 11
    slide = prs.slides.add_slide(prs.slide_layouts[1])
    #content=str(area)+'白名单机构投资事件分析'
    add_title(slide,content)
    add_square_title(slide,title)
    db['融资金额（亿￥）'] = db['投资模糊金额(万)']/10000
    db = db[['项目','企名片一级行业','业务','轮次','日期','融资金额（亿￥）','投资机构']]
    df = db.values
    df = np.insert(df, 0, values=np.array([['项目','标签','业务','轮次','时间','融资金额（亿￥）','投资机构']]), axis=0)
    # 设置表格位置和大小
    left, top, width, height = Cm(0.1), Cm(3.64), Cm(24.51), Cm(15.63)
    rows,cols = len(df),len(df[0])
    # 表格行列数，和大小
    shape = slide.shapes.add_table(rows, cols, left, top, width, height)
    #改表格样式
    tbl = shape._element.graphic.graphicData.tbl
    tbl[0][-1].text = '{3B4B98B0-60AC-42C2-AFA5-B58CD77FA1E5}'
    table = shape.table
    # table.apply_style = '浅色样式1-强调1'
    #赋值，调格式
    table.columns[0].width = Cm(2.2)
    table.columns[1].width = Cm(2.3)
    table.columns[2].width = Cm(4)
    table.columns[3].width = Cm(1.7)
    table.columns[4].width = Cm(2.5)
    table.columns[5].width = Cm(2.3)
    table.columns[6].width = Cm(10.2)

    for row in range(rows):
        table.rows[row].height= Cm(0.43)
        for col in range(cols):
            if (type(df[row][col]) == type(12.3)):
                if (df[row][col]>0)==False:
                    table.cell(row, col).text = ""
                else:
                    table.cell(row, col).text = format(df[row][col],'.0f')
            else:
                table.cell(row, col).text = str(df[row][col])
             # 设置文字大小
            table.cell(row, col).text_frame.paragraphs[0].font.size = Pt(font_size)
            # 设置字体
            table.cell(row, col).text_frame.paragraphs[0].font.name = '华文楷体'
            # table.cell(row, col).text_frame.paragraphs[0].font.name = 'Corbel'
            # 设置文字颜色
            # table.cell(row, col).text_frame.paragraphs[0].font.color.rgb = RGBColor(0, 0, 0)
            # 设置文字左右对齐
            table.cell(row, col).text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
            # 设置文字上下对齐
            table.cell(row, col).vertical_anchor = MSO_ANCHOR.MIDDLE
top10_list = list(pd.read_excel('D:\Desktop\钟鼎资本\代码\软件市场跟踪\所需文件\统计指标.xlsx')['名字'])[0:10]
prs = Presentation(r'D:\Desktop\钟鼎资本\代码\自动化PPT模版.pptx')
for i in range(0,2):
    for j in range(len(top10_list)):
        db = origin_data[origin_data['投资机构'].str.contains(top10_list[j],na = False)]
        if i == 0:#近一年
            db = abstract_time(db,'2020-06-02','2021-06-02')
            db = db.sort_values(by='投资模糊金额(万)',ascending=False).iloc[0:10]
            content = "近一年(2020.6-2021.6)所有软件项目数量Top 10机构Break Down（"+str(j+1)+"/10）："+top10_list[j]
            title = '近一年TOP10 Deal'
        else:#近三年
            content = "近三年(2018.6-2021.6)所有软件项目数量Top 10机构Break Down（"+str(j+1)+"/10）："+top10_list[j]
            db = abstract_time(db,'2018-06-02','2021-06-02')
            db = db.sort_values(by='投资模糊金额(万)',ascending=False).iloc[0:10]
            title = '近三年TOP10 Deal'
        db['日期'] = db['日期'].apply(date_to_ymd)
        get_software_expand_slide(db,content,title)
prs.save(r'D:\Desktop\软件top10展开.pptx')


In [ ]:
table_types = [
'{2D5ABB26-0587-4C30-8999-92F81FD0307C}''{3C2FFA5D-87B4-456A-9821-1D502468CF0F}''{284E427A-3D55-4303-BF80-6455036E1DE7}','{69C7853C-536D-4A76-A0AE-DD22124D55A5}','{775DCB02-9BB8-47FD-8907-85C794F793BA}','{35758FB7-9AC5-4552-8A53-C91805E547FA}','{08FB837D-C827-4EFA-A057-4D05807E0F7C}','{5940675A-B579-460E-94D1-54222C63F5DA}','{D113A9D2-9D6B-4929-AA2D-F23B5EE8CBE7}','{18603FDC-E32A-4AB5-989C-0864C3EAD2B8}','{306799F8-075E-4A3A-A7F6-7FBC6576F1A4}','{E269D01E-BC32-4049-B463-5C60D7B0CCD2}',
'{327F97BB-C833-4FB7-BDE5-3F7075034690}','{638B1855-1B75-4FBE-930C-398BA8C253C6}','{9D7B26C5-4107-4FEC-AEDC-1716B250A1EF}','{3B4B98B0-60AC-42C2-AFA5-B58CD77FA1E5}','{0E3FDE45-AF77-4B5C-9715-49D594BDF05E}','{C083E6E3-FA7D-4D7B-A595-EF9225AFEA82}','{D27102A9-8310-4765-A935-A1911B00CA55}','{5FD0F851-EC5A-4D38-B0AD-8093EC10F338}','{68D230F3-CF80-4859-8CE7-A43EE81993B5}','{7E9639D4-E3E2-4D34-9284-5A2195B3D0D7}','{69012ECD-51FC-41F1-AA8D-1B2483CD663E}','{72833802-FEF1-4C79-8D5D-14CF1EAF98D9}','{F2DE63D5-997A-4646-A377-4702673A728D}','{17292A2E-F333-43FB-9621-5CBBE7FDCDCB}','{5A111915-BE36-4E01-A7E5-04B1672EAD32}','{912C8C85-51F0-491E-9774-3900AFEF0FD7}','{616DA210-FB5B-4158-B5E0-FEB733F419BA}','{BC89EF96-8CEA-46FF-86C4-4CE0E7609802}','{5DA37D80-6434-44D0-A028-1B22A696006F}','{8799B23B-EC83-4686-B30A-512413B5E67A}','{ED083AE6-46FA-4A59-8FB0-9F97EB10719F}','{BDBED569-4797-4DF1-A0F4-6AAB3CD982D8}','{E8B1032C-EA38-4F05-BA0D-38AFFFC7BED3}','{793D81CF-94F2-401A-BA57-92F5A7B2D0C5}','{B301B821-A1FF-4177-AEE7-76D212191A09}','{9DCAF9ED-07DC-4A11-8D7F-57B35C25682E}','{1FECB4D8-DB02-4DC6-A0A2-4F2EBAE1DC90}','{1E171933-4619-4E11-9A3F-F7608DF75F80}','{FABFCF23-3B69-468F-B69F-88F6DE6A72F2}','{10A1B5D5-9B99-4C35-A422-299274C87663}','{073A0DAA-6AF3-43AB-8588-CEC1D06C72B9}','{5C22544A-7EE6-4342-B048-85BDC9FD1C3A}','{21E4AEA4-8DFA-4A89-87EB-49C32662AFE0}','{F5AB1C69-6EDB-4FF4-983F-18BD219EF322}','{00A15C55-8517-42AA-B614-E9B94910E393}','{7DF18680-E054-41AD-8BC1-D1AEF772440D}','{93296810-A885-4BE3-A3E7-6D5BEEA58F35}','{8EC20E35-A176-4012-BC5E-935CFFF8708E}','{6E25E649-3F16-4E02-A733-19D2CDBF48F0}','{85BE263C-DBD7-4A20-BB59-AAB30ACAA65A}','{EB344D84-9AFB-497E-A393-DC336BA19D2E}','{EB9631B5-78F2-41C9-869B-9F39066F8104}',
'{74C1A8A3-306A-4EB7-A6B1-4F7E0EB9C5D6}','{2A488322-F2BA-4B5B-9748-0D474271808F}','{D7AC3CCA-C797-4891-BE02-D94E43425B78}','{69CF1AB2-1976-4502-BF36-3FF5EA218861}','{8A107856-5554-42FB-B03E-39F5DBC370BA}','{0505E3EF-67EA-436B-97B2-0124C06EBD24}',
'{C4B1156A-380E-4F78-BDF5-A606A8083BF9}','{22838BEF-8BB2-4498-84A7-C5851F593DF1}','{16D9F66E-5EB9-4882-86FB-DCBF35E3C3E4}',
'{E8034E78-7F5D-4C2E-B375-FC64B27BC917}','{125E5076-3810-47DD-B79F-674D7AD40C01}','{37CE84F3-28C3-443E-9E96-99CF82512B78}','{D03447BB-5D67-496B-8E87-E561075AD55C}','{E929F9F4-4A8F-4326-A1B4-22849713DDAB}','{8FD4443E-F989-4FC4-A0C8-D5A2AF1F390B}','{AF606853-7671-496A-8E4F-DF71F8EC918B}','{5202B0CA-FC54-4496-8BCA-5EF66A818D29}','{0660B408-B3CF-4A94-85FC-2B1E0A45F4A2}',
'{91EBBBCC-DAD2-459C-BE2E-F6DE35CF9A28}','{46F890A9-2807-4EBB-B81D-B2AA78EC7F39}'
]
prs = Presentation()
title_slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(title_slide_layout)
title = slide.shapes.title
subtitle = slide.placeholders[1]

title.text = "python-pptx Table Format Examples"
subtitle.text = " "

for table_type in table_types:

    blank_slide_layout = prs.slide_layouts[5]
    slide = prs.slides.add_slide(blank_slide_layout)
    shapes = slide.shapes
    title_shape = shapes.title
    title_shape.text = table_type
    title_shape.text_frame.paragraphs[0].font.size=Pt(23)

    x, y, cx, cy = Inches(2.2), Inches(2.7), Inches(5.7), Inches(3.15)
    shape = slide.shapes.add_table(6, 9, x, y, cx, cy)

    """
    table styles
    https://github.com/scanny/python-pptx/issues/27
    """
    tbl =  shape._element.graphic.graphicData.tbl
    style_id = table_type
    tbl[0][-1].text = style_id

    table = shape.table

prs.save(r'D:\Desktop\tables.pptx')

In [ ]:
#IPO日期筛选
# start_time = dt.strptime('2021-05-26',"%Y-%m-%d").date()
# end_time = dt.strptime('2021-06-08',"%Y-%m-%d").date()
start_time = '2021-05-26'
end_time = '2021-06-08'
A_stock = pd.read_excel(r'D:\Desktop\钟鼎资本\代码\IPO数据\全部A股IPO.xlsx')
H_stock = pd.read_excel(r'D:\Desktop\钟鼎资本\代码\IPO数据\全部港股IPO.xlsx')
B_stock = pd.read_excel(r'D:\Desktop\钟鼎资本\代码\IPO数据\全部美股IPO.xlsx')

A_stock['上市日期'] = A_stock['上市日期'].astype(str)
H_stock['上市日期'] = H_stock['上市日期'].astype(str)
B_stock['上市日期'] = B_stock['上市日期'].astype(str)
A_stock[(A_stock['上市日期']>=start_time) & (A_stock['上市日期']<=end_time)].to_excel(r'D:\Desktop\钟鼎资本\代码\IPO数据\A股IPO.xlsx',index=False)
H_stock[(H_stock['上市日期']>=start_time) & (H_stock['上市日期']<=end_time)].to_excel(r'D:\Desktop\钟鼎资本\代码\IPO数据\港股IPO.xlsx',index=False)
B_stock[(B_stock['上市日期']>=start_time) & (B_stock['上市日期']<=end_time)].to_excel(r'D:\Desktop\钟鼎资本\代码\IPO数据\美股IPO.xlsx',index=False)




In [ ]:
#寻找首次进入

